In [1]:
import requests
import urllib3
from bs4 import BeautifulSoup

In [5]:
url = 'https://weather.kweather.co.kr/weather/life_weather'

response = requests.get(url, verify=False)
print(response.status_code)

200


D:\anaconda\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'weather.kweather.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
raw_data = response.text
soup = BeautifulSoup(raw_data)

title_list = []
titles = soup.select_one('div.life_weather_table').select('th')

for title in titles :
    title_list.append(title.text)
    
title_list

['나들이 지수',
 '빨래 지수',
 '세차 지수',
 '불조심 지수',
 '운동 지수',
 '공해 지수',
 '부패 지수',
 '자외선 지수',
 '냉방/난방 지수',
 '불쾌/감기 지수']

In [13]:
url = 'https://wwwnew.kweather.co.kr/forecast/forecast_living_jisu.html'

urllib3.disable_warnings()
urllib3.util.ssl_.DEFAULT_CIPHERS += ':HIGH:!DH:!aNULL'
try:
    urllib3.contrib.pyopenssl.util.ssl_.DEFAULT_CIPHERS += ':HIGH:!DH:!aNULL'
except AttributeError:
    # no pyopenssl support used / needed / available
    pass

response = requests.get(url, verify=False)
raw_data = response.text
soup = BeautifulSoup(raw_data)

jisu_navi_content = soup.select_one('ul.jisu_navi_content')
jisu_navi_content

<ul class="jisu_navi_content" id="jisu_navi_content">
<li>
<span id="area_1"><a href="javascript:startRequest('../data/JISU/11B00000.xml','1');"><b>서울/경기</b></a></span>
<span id="area_2"><a href="javascript:startRequest('../data/JISU/11D10000.xml','2');">강원영서</a></span>
<span id="area_3"><a href="javascript:startRequest('../data/JISU/11D20000.xml','3');">강원영동</a></span>
<span id="area_4"><a href="javascript:startRequest('../data/JISU/11C10000.xml','4');">충청북도</a></span>
<span id="area_5"><a href="javascript:startRequest('../data/JISU/11C20000.xml','5');">충청남도</a></span>
<span id="area_6"><a href="javascript:startRequest('../data/JISU/11H10000.xml','6');">경상북도</a></span>
<span id="area_7"><a href="javascript:startRequest('../data/JISU/11H20000.xml','7');">경상남도</a></span>
<span id="area_8"><a href="javascript:startRequest('../data/JISU/11F10000.xml','8');">전라북도</a></span>
<span id="area_9"><a href="javascript:startRequest('../data/JISU/11F20000.xml','9');">전라남도</a></span>
<span id="area_

In [57]:
code_list = []

for j in jisu_navi_content.select('a') :
    code_list.append(j['href'].split('U/')[1].split('\'')[0])

code_list

['11B00000.xml',
 '11D10000.xml',
 '11D20000.xml',
 '11C10000.xml',
 '11C20000.xml',
 '11H10000.xml',
 '11H20000.xml',
 '11F10000.xml',
 '11F20000.xml',
 '11G00000.xml']

In [58]:
xml_path = code_list[0]
xml_path

'11B00000.xml'

In [63]:
url = f'https://wwwnew.kweather.co.kr/data/JISU/{xml_path}'
response = requests.get(url)
response.encoding = 'utf-8'
text = response.text
soup = BeautifulSoup(text, 'html.parser')
soup

<?xml version='1.0' encoding='utf-8' ?>
<jisu>
<areacode>11B00000</areacode>
<areaname>서울,경기</areaname>
<tm>2023/12/08</tm>
<tmyyyy>2023</tmyyyy>
<tmmm>12</tmmm>
<tmdd>08</tmdd>
<tmyo>금</tmyo>
<ndate num="0">
<jtitle>
<name>나들이</name>
<jfix>나들이</jfix>
<num>01</num>
<jnum>80</jnum>
<jtext>나들이 즐길만 해요</jtext>
</jtitle>
<jtitle>
<name>빨래</name>
<jfix>빨래</jfix>
<num>02</num>
<jnum>50</jnum>
<jtext>미세먼지 한 때 많아요.</jtext>
</jtitle>
<jtitle>
<name>세차</name>
<jfix>세차</jfix>
<num>03</num>
<jnum>50</jnum>
<jtext>짙은 먼지만 닦아 내세요</jtext>
</jtitle>
<jtitle>
<name>난방</name>
<jfix>난방</jfix>
<num>04</num>
<jnum>50</jnum>
<jtext>활동시간에도 약간만</jtext>
</jtitle>
<jtitle>
<name>불조심</name>
<jfix>불조심</jfix>
<num>05</num>
<jnum>50</jnum>
<jtext>주의가 필요합니다</jtext>
</jtitle>
<jtitle>
<name>운동</name>
<jfix>운동</jfix>
<num>06</num>
<jnum>60</jnum>
<jtext>따뜻한 운동복을 준비하세요</jtext>
</jtitle>
<jtitle>
<name>감기</name>
<jfix>감기</jfix>
<num>07</num>
<jnum>40</jnum>
<jtext>그래도 조심하세요</jtext>
</jtitle>
<jtitle>
<name>공해</name>
<jfix

In [70]:
num = soup('num')
name = soup('name')
jnum = soup('jnum')
jtext = soup('jtext')

In [73]:
num_list = [n.text for n in num]
name_list = [n.text for n in name]
jnum_list = [n.text for n in jnum]
jtext_list = [n.text for n in jtext]

In [78]:
ndate_list = {code_list[0][:-4]:list(zip(num_list, name_list, jnum_list, jtext_list))}
ndate_list

{'11B00000': [('01', '나들이', '80', '나들이 즐길만 해요'),
  ('02', '빨래', '50', '미세먼지 한 때 많아요.'),
  ('03', '세차', '50', '짙은 먼지만 닦아 내세요'),
  ('04', '난방', '50', '활동시간에도 약간만'),
  ('05', '불조심', '50', '주의가 필요합니다'),
  ('06', '운동', '60', '따뜻한 운동복을 준비하세요'),
  ('07', '감기', '40', '그래도 조심하세요'),
  ('08', '공해', '70', '개인위생에 조금만 신경을'),
  ('09', '냉방', '20', '냉방 불필요해요'),
  ('10', '불쾌', '65', '기분 내세요'),
  ('11', '자외선', '30', '비교적 안전해요'),
  ('12', '모기', '30', '모기약 필요없어요'),
  ('13', '부패', '20', '음식보관 장소는 늘 청결하게'),
  ('33', '골프', '90', '라운딩에 최상의 조건입니다.'),
  ('01', '나들이', '30', '마스크를 반드시 착용하세요.'),
  ('02', '빨래', '30', '실내건조하세요.'),
  ('03', '세차', '40', '미세먼지 농도가 높아 다시 더러워져요'),
  ('04', '난방', '50', '적당한 난방이 필요해요'),
  ('05', '불조심', '50', '미리미리 조심하세요'),
  ('06', '운동', '30', '미세먼지 많아 운동 힘들어요.'),
  ('07', '감기', '40', '그래도 조심하세요'),
  ('08', '공해', '70', '개인위생에 조금만 신경을'),
  ('09', '냉방', '20', '냉방 불필요해요'),
  ('10', '불쾌', '65', '즐거운 하루 계획하세요'),
  ('11', '자외선', '30', '비교적 안전해요'),
  ('12', '모기', '30', '모기 걱정 없어요'),
  ('13', '부패',